# Homework 2: Naive Bayes
## Due September 19th

## Legal reasoning (From Murphy, 2.2). (25 pts)

Suppose a crime has been committed. Blood is found at the scene for which there is no innocent explanation. It is of a type which is present in 1% of the population.  The defendant is known to have this rare blood type.  The  prosecutor claims: “There is a 1% chance that the defendant would have the crime blood type if he were innocent. Thus there is a 99% chance that he guilty”. This is known as the prosecutor’s fallacy. What is wrong with this argument?

Alright you low rate prosecutor, let's talk about this fallacy you just made. 
Lets assume that testing the blood on the knife is an infallable test and no false positives ever, it wont change this argument.
- Its a true statement that 1% of the population DOES have this blood type. 
- Its true there is 1 murderer with this blood type out in the world somewhere, based on our assumed infallable test on the weapon.

What you probably meant is that given my client were innocent, there would only be a 1% chance that my client would match the blood sample:
$$P(\mathrm{Blood Match}|\mathrm{Innocence}) = \frac{1}{100} $$

But, what you claim __fallaciously__ is that;
$$P(\mathrm{Innocence}|\mathrm{Blood Match}) = \frac{1}{100} $$

Yet, that couldn't be further from the truth. Let's call S the set of everyone with any blood type. 99% of that set S is innocent for certain. That leaves 1% of S that share my client's blood type. 
1% of S, how many people is that? We could live in a TINY town with a population of 10000. 1% of 10000 is 100 people. Let's talk about the probability of you picking the right person in this tiny town based on the statistics you brought to this trial. 

Prosecutor, the probability you have the right person is; $$ \frac{1}{100} = 0.01 $$ in this tiny town.

An the probability you are pursuing the wrong person is;
$$ 1 - \frac{1}{100} = 0.99 $$

## Ham vs Spam (75 pts)
One use of the naive Bayes classifier, which is still in practical use today, is as a spam filter.  Consider the corpus of text messages packaged with this homework, which are each labelled as either 'spam' or 'ham'.  In this case, naive Bayes utilizes a Bernoulli model that quantifies the probability of a given word given that the message is either spam or ham.  For example, investigating the text messages here, we find that the word *draw* shows up in spam 27 times, implying that
$$P(X=\mathrm{draw}|Y=\mathrm{spam}) = \frac{27}{25748} = \frac{m_{draw,spam}}{m_{spam}},$$
while in the case of ham, it shows up 5 times so
$$P(X=\mathrm{draw}|Y=\mathrm{ham}) = \frac{5}{67148} = \frac{m_{draw,ham}}{m_{ham}}.$$
Thus we see that the word 'draw' shows up with a much higher frequency in spam e-mails than in ham.

While this is not particularly strong evidence on its own, we can create a powerful classifier by using the naive assumption in conjunction with all the words in a given message:
$$ P(Y=\mathrm{ham}|X=x) \propto P(Y=\mathrm{ham}) \prod_{i=1}^n P(X_i=x_i|Y=\mathrm{ham}), $$
$$ P(Y=\mathrm{spam}|X=x) \propto P(Y=\mathrm{spam}) \prod_{i=1}^n P(X_i=x_i|Y=\mathrm{spam}), $$
where $x_i$ are the words in a given message. 

Your task is to write such a classifier.  I have taken the somewhat tedious step of parsing the data for you, yielding the variables *word_dictionary*, which contains the ham and spam counts for each word, as well as *training_labels*, which provides the spam/ham labels for each text message.  I have also parsed a set of test data: *test_messages* is a list, each entry containing another list of the words in the text message, as well as *test_labels* which contains the spam/ham label for each message.

In [150]:
import numpy as np

# Maps from 'ham' or 'spam' strings to zero or one
def mapper(s):
    if s=='spam':
        return 0
    else:
        return 1

# Read in the text file
f = open('SMSSpamCollection','r')
lines = f.readlines()

# Break out the test data
test_lines = lines[:len(lines)//5]
lines = lines[len(lines)//5:]

# Instantiate the frequency dictionary and an array to
# record whether the line is ham or spam
word_dictionary = {}
training_labels = np.zeros(len(lines),dtype=int)

# Loop over all the training messages
for i,l in enumerate(lines):
    # Split into words
    l = l.lower().split()
    # Record the special first word which always ham or spam
    if l[0]=='ham':
        training_labels[i] = 1
   
    # For each word in the message, record whether the message was ham or spam
    for w in l[1:]:
        # If we've never seen the word before, add a new dictionary entry
        if w not in word_dictionary:
            word_dictionary[w] = [1,1]
        # If spam [word][+1, 0], if ham [word][0,+1]
        word_dictionary[w][mapper(l[0])] += 1
        
# Loop over the test messages
test_labels = np.zeros(len(test_lines),dtype=int)
test_messages = []
for i,l in enumerate(test_lines):
    l = l.lower().split()
    if l[0]=='ham':
        test_labels[i] = 1
    test_messages.append(l)

counts = np.array([v for v in word_dictionary.values()]).sum(axis=0)


Below, I have provided code skeletons. Your job is to make the code skeletons into an operational naive Bayes spam detector. (you may discard these skeletons if you would prefer to code this from scratch). Note that lines where you will need to change the code are marked with a '#!'.

Your first task is train the model:

In [40]:
#What is the prior P(Y=ham) ?

# ham messages / total messages = P(Y=ham)
ham_prior = sum(training_labels)/len(word_dictionary)
# Messages which are not ham are spam; the compliment ham
spam_prior = 1 - ham_prior

ham_prior 
spam_prior

def w_tot(val):
    return val[0] + val[1]

# What are the class probabilities P(X=word|Y=ham) for each word?

# P(X='word' | Y=spam) = (count 'word' found in spam)/(total word uses)
# P(X='word' | Y=ham) = (count of 'word' found in ham)/(total word uses) 

ham_likelihood = {}
spam_likelihood = {}
for key,val in word_dictionary.items():
    ham_likelihood[key] =  val[1] / w_tot(val)
    spam_likelihood[key] = val[0] / w_tot(val)
    
ham_likelihood['you']

0.868013468013468

Your next task is to make predictions on a set of test examples which were held back from the training procedure (see *test_messages* variable).  For each of these messages, compute the ham and spam probabilities.

In [128]:
# Where to hold the ham and spam posteriors
posteriors = np.zeros((len(test_lines),2))

# Loop over all the messages in the test set
for i,m in enumerate(test_messages):
    posterior_ham = 1.0
    posterior_spam = 1.0
    # Prior is multiplied into the posterior once
    posterior_ham *= ham_prior
    posterior_spam *= spam_prior
    
    # Loop over all the words in each message
    for w in m:
        # The purpose of this try/except handler is 
        # if we find a word which is not contained in the training data
        # we have to throw it out as it has no prior probability 
        # otherwise we have a bad probability for the new word. 
        try:
            posterior_ham *= 1 * ham_likelihood[w]
            posterior_spam *= 1 * spam_likelihood[w]
        except KeyError:
            pass
    
    
    # Notice the normalization factor (denominator) 
    # to turn these into proper probabilities!
    
    
    posteriors[i,0] = posterior_spam/(posterior_spam + posterior_ham)
    posteriors[i,1] = posterior_ham/(posterior_spam + posterior_ham)


1114

Finally, make a ham/spam prediction based on your posterior probabilities.  Compare these to the labels contained in test_labels.  Report the accuracy of your classifier as percentage correct.

In [149]:
texts_correct = 0
total_texts = len(posteriors)

len(test_labels)

for i,tlabel in enumerate(test_labels):
    # tlabel = 1 if the test label was marked 'ham'
    if (tlabel == 1) and (posteriors[i,0] < 0.5):
        texts_correct += 1 

classifier_accuracy = texts_correct / total_texts
classifier_accuracy


0.8456014362657092